In [1]:
!pip install requests beautifulsoup4 folium geopy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/108.4 kB ? eta -:--:--
   ----------- --------------------------- 30.7/108.4 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 108.4/108.4 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 125.4/125.4 kB 7.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/40.3 kB ? eta -:--:--
   ---------------------------------------- 40.3/40.3 kB ? eta 0:00:00


In [33]:
import requests
import folium
from geopy.geocoders import Nominatim
from datetime import datetime
import re
import random

# Replace 'your_api_key_here' with your actual News API key
api_key = '6ebe77095b7445689954c1336afb6fa0'
keyword = 'Donald Trump'
base_url = 'https://newsapi.org/v2/everything'

# Get today's date
today = datetime.now().strftime('%Y-%m-%d')

# Initialize geolocator
geolocator = Nominatim(user_agent="news_geolocator")

# Initialize map centered on USA
map = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

# Function to fetch news and plot locations on map
def fetch_and_plot_news(keyword, map):
    try:
        params = {
            'q': keyword,
            'apiKey': api_key,
            'pageSize': 100,  # Fetch up to 100 articles
        }
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            news_data = response.json()
            if news_data.get('articles'):
                city_counts = {}
                print('Total cities ',len(news_data['articles']))
                for article in news_data['articles']:
                    city_name = extract_city_from_article(article)
                    if city_name:
                        if city_name in city_counts:
                            city_counts[city_name] += 1
                        else:
                            city_counts[city_name] = 1
                        
                        location = geolocator.geocode(city_name)
                        if location:
                            # Add a small random offset to the latitude and longitude
                            lat_offset = random.uniform(-1, 1)
                            lon_offset = random.uniform(-1, 1)
                            
                            folium.Marker(
                                [location.latitude + lat_offset, location.longitude + lon_offset],
                                popup=f"<b>{article['title']}</b><br>{article['description']}"
                            ).add_to(map)
                
                # Print city counts
                print("City counts:")
                for city, count in city_counts.items():
                    print(f"{city}: {count} articles")
                
                # Check for cities with no data
                all_cities = set(city_counts.keys())
                cities_with_data = set(city_counts.keys())
                cities_with_no_data = all_cities - cities_with_data
                if cities_with_no_data:
                    print("Cities with no data:")
                    for city in cities_with_no_data:
                        print(city)
            else:
                print("No articles found for the keyword.")
        else:
            print(f"Error fetching news. Status code: {response.status_code}")
    except Exception as e:
        print(f"Error fetching or plotting news: {e}")

# Function to extract city from article content
def extract_city_from_article(article):
    try:
        if 'content' in article and article['content']:
            text = article['content']
            # Regular expression pattern to find city names
            city_pattern = r'\b(?:New York|Los Angeles|Chicago|Houston|Phoenix|Philadelphia|San Antonio|San Diego|Dallas|San Jose|Austin|Jacksonville|Fort Worth|Columbus|San Francisco|Charlotte|Indianapolis|Seattle|Denver|Washington|Boston|El Paso|Nashville|Detroit|Oklahoma City|Portland|Las Vegas|Memphis|Louisville|Baltimore)\b'
            
            # Find all cities in the text using the pattern
            cities_found = re.findall(city_pattern, text, flags=re.IGNORECASE)
            
            if cities_found:
                # Return the first city found (you can modify as needed for your use case)
                return cities_found[0]
        
        return None
    except Exception as e:
        print(f"Error extracting city from article: {e}")
        return None

# Call the function to fetch and plot news
fetch_and_plot_news(keyword, map)

# Save the map to an HTML file
map.save("us_news_map.html")

print("Map generation complete. Check us_news_map.html for the map.")


Total cities  100
City counts:
Washington: 5 articles
Nashville: 1 articles
WASHINGTON: 1 articles
Philadelphia: 1 articles
New York: 1 articles
Map generation complete. Check us_news_map.html for the map.
